# <font color=darkgreen>*Sentinel P-Type Dataset*</font>
---

### **Imports**

In [1]:
%matplotlib inline

# core
import os
import sys
import glob
import math
import cfgrib
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib as mpl

# netCDF4
import netCDF4 as nc
from netCDF4 import Dataset

# datetime
from datetime import datetime,timedelta

#cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeat

# Warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# matplotlib
import matplotlib.colors as mcolors
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib import colors
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.dates import DateFormatter, AutoDateLocator,HourLocator,DayLocator,MonthLocator

# metpy
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import StationPlot
from metpy.calc import wind_speed, wind_direction, relative_humidity_from_dewpoint, wind_components
from metpy.calc import wind_speed, wind_direction, relative_humidity_from_dewpoint, wind_components

### **Merge Data**

In [2]:
import xarray as xr
# Canadian Sentinal Network Data
sent_base = '/network/rit/home/je845911/minlab/je845911/data/sentinal/sentinels_metdata.nc'
sent_ice = '/network/rit/home/je845911/minlab/je845911/data/sentinal/p-type/CFI_Sentinels_icing_detector_data_WINTRE-MIX.nc' 
sent_precip = '/network/rit/home/je845911/minlab/je845911/data/sentinal/p-type/sentinels_hotplates.nc'
sent_snow = '/network/rit/home/je845911/minlab/je845911/data/sentinal/p-type/CFI_Sentinels_Snowdepth.nc' 
troi_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl78363/TROI_Parsivel_WINTRE-MIX.nc'
abor_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl29537/CFI_Sentinels_Arboretum_Parsivel_Disdrometer_data_WINTRE-MIX_02.nc'
uqam_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl34331/UQAM_Parsivel_WINTRE-MIX.nc'
gaul_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl70655/CFI_Sentinels_Gault_Parsivel_Disdrometer_data_WINTRE-MIX_03.nc'

In [3]:
# Canadian Sentinal Network Data
sent_base = '/network/rit/home/je845911/minlab/je845911/data/sentinal/sentinels_metdata.nc'
sent_ice = '/network/rit/home/je845911/minlab/je845911/data/sentinal/p-type/CFI_Sentinels_icing_detector_data_WINTRE-MIX.nc' 
sent_snow = '/network/rit/home/je845911/minlab/je845911/data/sentinal/p-type/CFI_Sentinels_Snowdepth.nc' 
sent_precip = '/network/rit/home/je845911/minlab/je845911/data/sentinal/p-type/sentinels_hotplates.nc'
troi_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl78363/TROI_Parsivel_WINTRE-MIX.nc'
abor_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl29537/CFI_Sentinels_Arboretum_Parsivel_Disdrometer_data_WINTRE-MIX_02.nc'
uqam_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl34331/UQAM_Parsivel_WINTRE-MIX.nc'
gaul_dis = '/network/rit/home/je845911/minlab/je845911/data/sentinal/downloads/jengl70655/CFI_Sentinels_Gault_Parsivel_Disdrometer_data_WINTRE-MIX_03.nc'
varia = ['station','time']

# Rainfall Intensity 
t_dis = xr.open_dataset(troi_dis)
t_dis = t_dis.drop(['latitude','longitude','height_above_mean_sea_level','bin_velocities_width','bin_diameters_width','Sample_Int','Num_Particles','Sensor_Temp','Raw_Data','Nd_Spectra','Vd_Spectra'])
t_dis = t_dis.drop_dims(['bin_velocities','bin_diameters'])
t_dis = t_dis.to_dataframe()
t_dis = t_dis.reset_index(level=0)
t_dis['station']= 'TROI'
#t_dis = t_dis.set_index(varia)

a_dis = xr.open_dataset(abor_dis)
a_dis = a_dis.drop(['latitude','longitude','height_above_mean_sea_level','bin_velocities_width','bin_diameters_width','Prcp_Start','Wx_Code_Synop','Wx_Code_METAR','Wx_Code_NWS','Reflectivity','Visibility','Signal_Ampl','Num_Particles','Sensor_Temp','Heating_Current','Sensor_Voltage','Kinetic_Energy','Snow_Intensity','Raw_Data','Nd_Spectra','Vd_Spectra'])
a_dis = a_dis.drop_dims(['bin_velocities','bin_diameters'])
a_dis = a_dis.to_dataframe()
a_dis = a_dis.reset_index(level=0)
a_dis['station']= 'ARBO'
#a_dis = a_dis.set_index(varia)

g_dis = xr.open_dataset(gaul_dis)
g_dis = g_dis.drop(['latitude','longitude','height_above_mean_sea_level','bin_velocities_width','bin_diameters_width','Prcp_Start','Wx_Code_Synop','Wx_Code_METAR','Wx_Code_NWS','Reflectivity','Visibility','Signal_Ampl','Num_Particles','Sensor_Temp','Heating_Current','Sensor_Voltage','Kinetic_Energy','Snow_Intensity','Raw_Data','Nd_Spectra','Vd_Spectra'])
g_dis = g_dis.drop_dims(['bin_velocities','bin_diameters'])
g_dis = g_dis.to_dataframe()
g_dis = g_dis.reset_index(level=0)
g_dis['station']= 'GAUL'
#g_dis = g_dis.set_index(varia)


u_dis = xr.open_dataset(uqam_dis)
u_dis = u_dis.drop(['latitude','longitude','height_above_mean_sea_level','bin_velocities_width','bin_diameters_width','Prcp_Start','Wx_Code_Synop','Wx_Code_METAR','Wx_Code_NWS','Reflectivity','Visibility','Signal_Ampl','Num_Particles','Sensor_Temp','Heating_Current','Sensor_Voltage','Kinetic_Energy','Snow_Intensity','Raw_Data','Nd_Spectra','Vd_Spectra'])
u_dis = u_dis.drop_dims(['bin_velocities','bin_diameters'])
u_dis = u_dis.to_dataframe()
u_dis = u_dis.reset_index(level=0)
u_dis['station']= 'UQAM'
#u_dis = u_dis.set_index(varia)

sent_pcp = pd.concat([g_dis,t_dis,a_dis,u_dis])

In [4]:
# Temperature
sent_base = xr.open_dataset(sent_base)
sent_base = sent_base.drop(['accumulated_precipitation','max_wind_direction','height_above_mean_sea_level','relative_humidity','station_pressure','wind_speed','wind_direction','wind_direction_stddev','max_wind_speed','max_wind_direction'])
sent_base = sent_base.to_dataframe()
sent_base = sent_base.reset_index(level=1)
sent_base = sent_base.rename(columns={"latitude":"lat","longitude":"lon"})

In [5]:
# Ice Accumulation
sent_ice = xr.open_dataset(sent_ice, drop_variables = 'station')
sent_ice = sent_ice.drop(['heater_status','ice','height_above_mean_sea_level'])
new_time, empty_time = [], []
sentinal_time, mock_time = datetime(2021,11,1,0,0), range(0,217440,1)
for i in mock_time:
    adj_time = sentinal_time + timedelta(minutes=i)
    new_time.append(adj_time)
time = np.array(new_time)
sent_ice = sent_ice.assign_coords({"time":time})
sent_ice = sent_ice.to_dataframe()
sent_ice = sent_ice.reset_index(level=1)
sent_ice['station'] = sent_ice['station'].replace([0], 'GAUL')
sent_ice['station'] = sent_ice['station'].replace([1], 'ARBO')
sent_ice['station'] = sent_ice['station'].replace([2], 'UQAM')
sent_ice['station'] = sent_ice['station'].replace([3], 'TROI')
sent_ice.loc[sent_ice['lon'] < 0, 'lon'] = sent_ice['lon'] - sent_ice['lon'] - sent_ice['lon'] # Change Lon values from - to +

In [6]:
# Snow Accumulation
sent_snow = xr.open_dataset(sent_snow)
sent_snow = sent_snow.drop(['height_above_mean_sea_level','swe_cs725'])
new_time, empty_time = [], []
sentinal_time, mock_time = datetime(2021,11,1,0,0), range(0,216001,1)
for i in mock_time:
    adj_time = sentinal_time + timedelta(minutes=i)
    new_time.append(adj_time)
time = np.array(new_time)
sent_snow = sent_snow.assign_coords({"time":time})
sent_snow = sent_snow.to_dataframe()
sent_snow = sent_snow.reset_index(level=1)
sent_snow = sent_snow.rename(columns={"latitude":"lat","longitude":"lon"})
sent_snow.loc[sent_snow['lon'] < 0, 'lon'] = sent_snow['lon'] - sent_snow['lon'] - sent_snow['lon'] # Change Lon values from - to +

In [7]:
# Base Precipitation Changes
sent_ap = xr.open_dataset(sent_precip)
sent_ap = sent_ap.to_dataframe()
sent_ap = sent_ap.rename(columns={"latitude":"lat","longitude":"lon"})
sent_ap = sent_ap.drop(columns=['height_above_mean_sea_level','status_k63'])

In [8]:
# Merge All Data
sent_total = pd.merge(sent_ice, sent_base, on=["time","lat","lon"])
sent_total = pd.merge(sent_total, sent_snow, on=['time',"station"])
sent_total = pd.merge(sent_total, sent_ap, on=['time',"station"])
sent_total = sent_total.drop(columns=['lat_y','lat','lon_y','lon'])
sent_total = sent_total.rename(columns={"lat_x":"lat","lon_x":"lon"})
sent_total = sent_total.set_index(['time','station'])
sent_total = sent_total.loc[('2022-02-01 00:00:00'):('2022-03-15 00:00:00')]

sent_pcp_adj = sent_pcp.set_index(['time','station'])
sent_total = pd.merge(sent_total, sent_pcp_adj, on=["time","station"])

In [9]:
# Add Ice Coversion becasue I forgot earlier.
sent_total['ice_status'] = sent_total['ice_status'].replace([b'Y'], 1)
sent_total['ice_status'] = sent_total['ice_status'].replace([b'N'], 0)
sent_total['ice_status'] = sent_total['ice_status'].replace([b'E'], 0)
sent_total['ice_status'] = sent_total['ice_status'].replace([b'M'], 0)
sent_total['ice_status'] = sent_total['ice_status'].replace([b'Q'], 0)
sent_total['ice_status'] = sent_total['ice_status'].replace([b'O'], 0)

# Fixing Snow periods
sent_total['snow_depth_avg_sdms40'] = sent_total['snow_depth_avg_sdms40'].diff(periods = 4)
sent_total['snow_depth_sr50a'] = sent_total['snow_depth_sr50a'].diff(periods = 4)

# Fixing Duplicate index issue (took 12 hours) :()
sent_test = sent_total[~sent_total.index.duplicated(keep='first')]

In [10]:
xra = sent_test.to_xarray()
xra['ice_status'].astype(int)
xr_ice= xra.resample(time='15min').max(dim='time')
xra = xra.resample(time='15min').mean(dim='time')

for i in xra['station'].values:

    # P_TYPE NO PRECIP. CHECK (0.0004)
    a = np.where(xra['Prcp_Intensity'].sel(station = i) > 0.125,-0.1,0)#0.025
    #k = np.where(xr['precipitation_rate_k63'].sel(station = i) == 'nan',13.99,0)#0.025

    # P-TYPE ICE CHECK (0.025)
    b = np.where((xra['Prcp_Intensity'].sel(station = i) > 0.125) & (xra['temp_2m'].sel(station = i) <= 0) & (xr_ice['ice_status'].sel(station = i) == 1),3.99,a)

    # P-TYPE RAIN CHECK (0.025)
    c = np.where((xra['Prcp_Intensity'].sel(station = i) > 0.125) &(xra['temp_2m'].sel(station = i) > 0) & (xr_ice['ice_status'].sel(station = i) == 0),9.99,b)

    # P-TYPE SNOW CHECK (0.025)
    d = np.where((xra['Prcp_Intensity'].sel(station = i) > 0.125) & (xra['temp_2m'].sel(station = i) <= 0) & (xra['snow_depth_sr50a'].sel(station = i) >= 1.25) & (xr_ice['ice_status'].sel(station = i) == 0),0.99,c)

    # P-TPYE SNOW CHECK (0.025)
    e = np.where((xra['Prcp_Intensity'].sel(station = i) > 0.125) & (xra['temp_2m'].sel(station = i) <= 0) & (xra['snow_depth_avg_sdms40'].sel(station = i) >= 1.25) & (xr_ice['ice_status'].sel(station = i) == 0),0.99,d)

    # ASOS Mapping
    #precip_types_map = {'NULL': -0.1,'Snow and/or Graupel': 0.99,'Ice Pellets/Sleet': 1.99,'Mixed Ice Pellets and Snow': 2.99,'Freezing Rain': 3.99,'Freezing Drizzle': 3.99,'Mixed Freezing Rain and Ice Pellets': 6.99,'Rain': 9.99, 'Drizzle': 9.99,'Mixed Rain and Snow': 10.99,'Mixed Rain and Ice Pellets': 11.99,'Unknown Precip': 13.99}

    # P-TYPE ICE CHECK (2)
    PTYPE = np.where((e==-0.1),13.99,e)
    
    if i == 'ARBO':
        PTYPE_ARBO = PTYPE
    elif i == 'TROI':
        PTYPE_TROI = PTYPE
    elif i == 'UQAM':
        PTYPE_UQAM = PTYPE
    elif i == 'GAUL':
        PTYPE_GAUL = PTYPE

In [11]:
c1 = xra['time'].to_dataframe()
c1['ptype'] = PTYPE_ARBO
c1['station']= 'ARBO'
c1 = c1.set_index(['time','station'])
c1 = c1.to_xarray()

c2 = xra['time'].to_dataframe()
c2['ptype'] = PTYPE_GAUL
c2['station']= 'GAUL'
c2 = c2.set_index(['time','station'])
c2 =c2.to_xarray()

c3 = xra['time'].to_dataframe()
c3['ptype'] = PTYPE_UQAM
c3['station']= 'UQAM'
c3 = c3.set_index(['time','station'])
c3 = c3.to_xarray()

c4 = xra['time'].to_dataframe()
c4['ptype'] = PTYPE_TROI
c4['station']= 'TROI'
c4 = c4.set_index(['time','station'])
c4 = c4.to_xarray()

ds_new = xr.concat([c1, c2],dim='station')
ds_new = xr.concat([ds_new, c4],dim='station')
ds_new = xr.concat([ds_new, c3],dim='station')
final = xr.merge([ds_new, xra])

### **Add Metadata**

In [173]:
# DataArray Metadata
final_ = final.assign_attrs(
    Contact = 'CFICS Precipitation Type (ptype) derived by John England, UAlbany, jengland@albany.edu')

# Time Metadata
final_['time'] = final_['time'].assign_attrs(
    Description = 'Time from 2021-11-01 00:00:00 to 2022-04-01 00:00:00',
    Format = 'YYYY-MM-DD HH:mm:SS')
    
# Station Metadata
final_['station'] = final_['station'].assign_attrs(
    Description = 'ARBO: Instrument yard of Arboretum CFI sentinel weather station.' + '\n' +
    'GAUL: Instrument yard of Gault CFI sentinel weather station, Mont St-Hilaire, QC, CAN.' + '\n' +
    'TROI: Instrument yard of Trois-Rivières CFI sentinel weather station, Trois-Rivières, QC, CAN.' + '\n' +
    'UQAM: UQAM-PK CFI sentinel weather station, rooftop of President-Kennedy building, Montreal, QC, CAN')

# Ice Metadata
final_['ice_status'] = final_['ice_status'].assign_attrs(
    Description = 'Ice Accumulation (Boolean)',
    Legend = 'Yes: 1, No: 0',
    Instrument = '0872F1 - Ice Detector (2400 BAUD Output)',
    Note = 'Re-sampled into 15 minute maximum values')
    
# Temperature Metadata
final_['temp_2m'] = final_['temp_2m'].assign_attrs(
    Description = '2-Meter Temperature in Deg. C',
    Instrument = 'HMP 155',
    Note = 'Re-sampled into 15 minute mean values')

# Accumulated Precipitation Metadata
final_['Prcp_Intensity'] = final_['Prcp_Intensity'].assign_attrs(
    Description = 'Accumulated Precipitation (mm)',
    Instrument = 'OTT Parsivel2 laser disdrometer',
    Note = 'Re-sampled into 15 minute mean values')

# Snow Depth (sdms40) Metadata
final_['snow_depth_avg_sdms40'] = final_['snow_depth_avg_sdms40'].assign_attrs(
    Description = 'Snowdepth Average measured by SDMS40 at ARBO, GAUL and UQAM.',
    Instrument = 'SDMS40 Multipoint Scanning Snowfall Sensor (for stations GAUL, ARBO and UQAM).',
    Note = 'Re-sampled into 15 minute mean values')

# Snow Depth (sr50a) Metadata
final_['snow_depth_sr50a'] = final_['snow_depth_sr50a'].assign_attrs(
    Description = 'Snowdepth measured by SR50A at GAUL and TROI. Minute data.',
    Instrument = 'SR50A Sonic Distance Sensor (for stations GAUL and TROI).',
    Note = 'Re-sampled over 15-min mean')

# P-type Metadata
final_['ptype'] = final_['ptype'].assign_attrs(
    Description = 'Derived precipitation type (p-type).',
    Legend = 'NULL: -0.1' + '\n' + 'Snow and/or Graupel: 0.99' + '\n' +
    'Ice Pellets/Sleet: 1.99' + '\n' + 'Mixed Ice Pellets and Snow: 2.99' + '\n' + 'Freezing Rain: 3.99' + '\n' + 'Freezing Drizzle: 3.99' + '\n' + 'Mixed Freezing Rain and Ice Pellets: 6.99' + '\n' + 'Rain: 9.99' + '\n' + 'Drizzle: 9.99' + '\n' + 'Mixed Rain and Snow: 10.99' + '\n' + 'Mixed Rain and Ice Pellets: 11.99' + '\n' + 'Unknown Precip: 13.99',
    Methodology = 'Re-sampled CFICS data for 15-minute intervals where p-type is derived from the following: 15-min. Averaged Temperature, 15-min. Averaged Accumulated Snowfall, 15-min. Averaged Accumulated Precipitation, and ice accumulation.')

final_ = final_.drop_vars(["lat","lon","precipitation_rate_k63"])
final_ = final_.rename({'ptype':'p-type','ice_status':'accu_ice','Prcp_Intensity':'accu_precip'})

In [182]:
final_.to_netcdf(path='v1_output')

###### England, John | LU20230317